In [2]:
from pandas import DataFrame, Series
import pandas as pd
import pyodbc
import re

In [3]:
path = "C:\Data\Project1CrimeData\Population\SAPE20DT1-mid-2017-lsoa-syoa-estimates-formatted.XLS"

In [4]:
sheet_names = pd.ExcelFile(path).sheet_names

In [6]:
sheet_names

['Contents',
 'Terms and conditions',
 'Notes and definitions',
 'Mid-2017 Persons',
 'Mid-2017 Males',
 'Mid-2017 Females',
 'Related Publications']

In [5]:
full_data = pd.read_excel(path, sheet_name='Mid-2017 Persons', header=3)

#remove aggregated area rows
lsoa_data = full_data[pd.isna(full_data['Area Names'])]

In [6]:
lsoa_data.head()

,Area Codes,Area Names,Unnamed: 2,All Ages,0,1,2,3,4,5,...,81,82,83,84,85,86,87,88,89,90+
1,E01020634,NaN,County Durham 001A,1632,19,13,16,15,23,19,...,7,8,10,8,9,3,1,9,4,7
2,E01020635,NaN,County Durham 001B,1329,18,13,25,12,20,10,...,8,7,4,7,6,5,5,7,3,10
3,E01020636,NaN,County Durham 001C,1725,19,10,26,20,17,27,...,9,11,8,6,9,6,4,2,2,14
4,E01020654,NaN,County Durham 001D,1826,9,17,14,16,13,16,...,16,9,9,10,14,12,12,9,11,31
5,E01020676,NaN,County Durham 001E,1517,12,13,18,18,13,14,...,8,3,3,4,5,7,4,0,1,3


In [7]:
lsoa_data.shape

(34753, 95)

In [8]:
ages = list(lsoa_data.columns)[4:]
age_labels = ['0-15', '16-30', '31-45', '45-60', '61-75', '76+']
age_bins = [-1,15, 30, 45, 60, 75, 100]

In [9]:
lsoa_unpiv = pd.melt(lsoa_data, id_vars=['Area Codes', 'Unnamed: 2'], value_vars=ages)
lsoa_unpiv = lsoa_unpiv.rename(index=str, columns={"Unnamed: 2": "Area Names", "variable": "Age", "value": "Population"})

lsoa_unpiv['age_for_binning'] = lsoa_unpiv['Age'].map(lambda x: 91 if x == '90+' else int(x))

lsoa_unpiv['age_bin'] = pd.cut(lsoa_unpiv.age_for_binning, bins=age_bins, labels=age_labels)

In [13]:
lsoa_unpiv.head()

,Area Codes,Area Names,Age,Population,age_for_binning,age_bin
0,E01020634,County Durham 001A,0,19,0,0-15
1,E01020635,County Durham 001B,0,18,0,0-15
2,E01020636,County Durham 001C,0,19,0,0-15
3,E01020654,County Durham 001D,0,9,0,0-15
4,E01020676,County Durham 001E,0,12,0,0-15


In [14]:
binned_lsoa_ages = lsoa_unpiv.groupby(['Area Names', 'age_bin']).sum()[['Population']]